## Cox Proportional Hazards and Random Survival Forests

We'll be using a dataset with survival data of patients with Primary Biliary Cirrhosis (pbc). PBC is a progressive disease of the liver caused by a buildup of bile within the liver (cholestasis) that results in damage to the small bile ducts that drain bile from the liver.

In [3]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lifelines import CoxPHFitter
from lifelines.utils import concordance_index as cindex
from sklearn.model_selection import train_test_split

from assignments.C2_W4.util import load_data

In [5]:
df = load_data()

In [6]:
print(df.shape)

(258, 19)


In [9]:
i = 2
df.iloc[i, :]

time          2.772603
status        1.000000
trt           0.000000
age          70.072553
sex           1.000000
ascites       0.000000
hepato        0.000000
spiders       0.000000
edema         0.500000
bili          1.400000
chol        176.000000
albumin       3.480000
copper      210.000000
alk.phos    516.000000
ast          96.100000
trig         55.000000
platelet    151.000000
protime      12.000000
stage         4.000000
Name: 2, dtype: float64

In [11]:
df_dev, df_test = train_test_split(df, test_size = 0.25)
df_train, df_val = train_test_split(df_dev, test_size=0.25)

print("Total number of patients:", df.shape[0])
print("Total number of patients in training set:", df_train.shape[0])
print("Total number of patients in validation set:", df_val.shape[0])
print("Total number of patients in test set:", df_test.shape[0])

Total number of patients: 258
Total number of patients in training set: 144
Total number of patients in validation set: 49
Total number of patients in test set: 65


### Cox Proportional Hazards

In [15]:
def to_one_hot(data, cols):
    one_hot_df = pd.get_dummies(data, columns=cols, drop_first=True, dtype=np.float64)
    
    return one_hot_df

In [16]:
to_encode = ['edema', 'stage']

one_hot_train = to_one_hot(df_train, to_encode)
one_hot_val = to_one_hot(df_val, to_encode)
one_hot_test = to_one_hot(df_test, to_encode)

print(one_hot_val.columns.tolist())
print(f"There are {len(one_hot_val.columns)} columns")

['time', 'status', 'trt', 'age', 'sex', 'ascites', 'hepato', 'spiders', 'bili', 'chol', 'albumin', 'copper', 'alk.phos', 'ast', 'trig', 'platelet', 'protime', 'edema_0.5', 'edema_1.0', 'stage_2.0', 'stage_3.0', 'stage_4.0']
There are 22 columns


In [17]:
print(one_hot_train.shape)
one_hot_train.head()

(144, 22)


,time,status,trt,age,sex,ascites,hepato,spiders,bili,chol,...,alk.phos,ast,trig,platelet,protime,edema_0.5,edema_1.0,stage_2.0,stage_3.0,stage_4.0
86,0.542466,1.0,0.0,37.278576,0.0,0.0,0.0,0.0,1.1,345.0,...,1860.0,218.55,72.0,447.0,10.7,0.0,0.0,0.0,1.0,0.0
247,4.890411,0.0,1.0,55.416838,0.0,0.0,1.0,0.0,0.8,324.0,...,1237.0,66.65,146.0,371.0,10.0,0.0,0.0,0.0,1.0,0.0
30,10.517808,1.0,1.0,41.552361,0.0,0.0,1.0,0.0,4.7,296.0,...,9933.2,206.40,101.0,195.0,10.3,0.0,0.0,1.0,0.0,0.0
212,5.967123,0.0,0.0,50.203970,0.0,0.0,0.0,1.0,0.5,400.0,...,1134.0,96.10,55.0,356.0,10.2,0.0,0.0,0.0,1.0,0.0
134,8.630137,0.0,0.0,42.967830,0.0,0.0,0.0,0.0,0.4,263.0,...,836.0,74.40,121.0,445.0,11.0,0.0,0.0,1.0,0.0,0.0


### Fitting and Interpreting a Cox Model

In [19]:
cph = CoxPHFitter()
cph.fit(one_hot_train, duration_col="time", event_col='status', step_size=0.1)

R:\Work\Anacond\lib\site-packages\lifelines\fitters\coxph_fitter.py:1598: ConvergenceWarning: Newton-Rhaphson convergence completed successfully but norm(delta) is still high, 0.807. This may imply non-unique solutions to the maximum likelihood. Perhaps there is collinearity or complete separation in the dataset?

  exceptions.ConvergenceWarning,


<lifelines.CoxPHFitter: fitted with 144 total observations, 92 right-censored observations>

In [20]:
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 144 total observations, 92 right-censored observations>
             duration col = 'time'
                event col = 'status'
      baseline estimation = breslow
   number of observations = 144
number of events observed = 52
   partial log-likelihood = -180.52
         time fit was run = 2021-09-17 08:45:33 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
trt        -0.28       0.76       0.35            -0.97             0.41                 0.38                 1.51
age         0.02       1.02       0.02            -0.02             0.06                 0.98                 1.06
sex         0.48       1.62       0.62            -0.74             1.70                 0.48                 5.46
ascites    -0.08       0.92       0.66            -1.37             1.21                 0.25                 3.35
hepato     -0.40       0.67       0.39            -1.18             0.37                 0.31                 1.45
spiders     0.76       2.13       0.38             0.01             1.50                 1.01                 4.50
bili        0.09       1.10       0.04             0.01             0.17                 1.01                 1.19
chol        0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
albumin    -0.45       0.64       0.45            -1.33             0.44                 0.26                 1.55
copper      0.00       1.00       0.00            -0.00             0.01                 1.00                 1.01
alk.phos    0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
ast         0.01       1.01       0.00             0.00             0.01                 1.00                 1.01
trig       -0.00       1.00       0.00            -0.01             0.00                 0.99                 1.00
platelet    0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
protime     0.35       1.42       0.19            -0.01             0.72                 0.99                 2.05
edema_0.5   0.10       1.11       0.47            -0.82             1.03                 0.44                 2.79
edema_1.0   0.56       1.75       0.64            -0.70             1.82                 0.50                 6.16
stage_2.0  17.08   2.62e+07    3133.06         -6123.61          6157.77                 0.00                  inf
stage_3.0  17.05   2.54e+07    3133.06         -6123.64          6157.75                 0.00                  inf
stage_4.0  18.12   7.37e+07    3133.06         -6122.58          6158.81                 0.00                  inf

              z    p   -log2(p)
covariate                      
trt       -0.79 0.43       1.23
age        0.77 0.44       1.17
sex        0.78 0.44       1.19
ascites   -0.12 0.90       0.15
hepato    -1.03 0.31       1.71
spiders    1.98 0.05       4.40
bili       2.30 0.02       5.55
chol       0.08 0.94       0.09
albumin   -0.99 0.32       1.63
copper     0.32 0.75       0.42
alk.phos   1.26 0.21       2.26
ast        2.09 0.04       4.79
trig      -0.65 0.52       0.95
platelet   0.94 0.35       1.52
protime    1.89 0.06       4.08
edema_0.5  0.22 0.83       0.28
edema_1.0  0.87 0.39       1.37
stage_2.0  0.01 1.00       0.01
stage_3.0  0.01 1.00       0.01
stage_4.0  0.01 1.00       0.01
---
Concordance = 0.86
Partial AIC = 401.04
log-likelihood ratio test = 89.36 on 20 df
-log2(p) of ll-ratio test = 33.28

### Hazard Ratio

In [21]:
def hazard_ratio(case_1, case_2, cox_params):
    hr = np.exp(cox_params.dot((case_1 - case_2).T))
    
    return hr

In [22]:
i = 1
case_1 = one_hot_train.iloc[i, :].drop(['time', 'status'])

j = 5
case_2 = one_hot_train.iloc[j, :].drop(["time", 'status'])

print(hazard_ratio(case_1.values, case_2.values, cph.params_.values))

0.43529834854111266


In [23]:
i = 4
case_a = one_hot_train.iloc[i, :].drop(['time', 'status'])

j = 7
case_b = one_hot_train.iloc[j, :].drop(['time', 'status'])

print("Case A\n\n", case_a, "\n")
print("Case B\n\n", case_b, "\n")
print("Hazard Ratio:", hazard_ratio(case_a.values, case_b.values, cph.params_.values))

Case A

 trt            0.00000
age           42.96783
sex            0.00000
ascites        0.00000
hepato         0.00000
spiders        0.00000
bili           0.40000
chol         263.00000
albumin        3.57000
copper       123.00000
alk.phos     836.00000
ast           74.40000
trig         121.00000
platelet     445.00000
protime       11.00000
edema_0.5      0.00000
edema_1.0      0.00000
stage_2.0      1.00000
stage_3.0      0.00000
stage_4.0      0.00000
Name: 134, dtype: float64 

Case B

 trt             0.000000
age            46.154689
sex             0.000000
ascites         0.000000
hepato          0.000000
spiders         0.000000
bili            1.600000
chol          325.000000
albumin         3.690000
copper         69.000000
alk.phos     2583.000000
ast           142.000000
trig          140.000000
platelet      284.000000
protime         9.600000
edema_0.5       0.000000
edema_1.0       0.000000
stage_2.0       0.000000
stage_3.0       1.000000
stage_4.0       0.0

### Harrell's C-index

In [32]:
def harrell_c(y_true, scores, event):
    n = len(y_true)
    
    concordant = 0.0
    permissible = 0.0
    ties = 0.0
    
    result = 0.0
    
    for i in range(n):
        for j in range(i+1, n):
            
            if event[i] == 1 or event[j] == 1:
                if event[i] == 1 and event[j] == 1:
                    permissible += 1
                    
                    if scores[i] == event[j]:
                        ties += 1
                    elif y_true[i] < y_true[j] and scores[i] > scores[j]:
                        concordant += 1
                    elif y_true[i] > y_true[j] and scores[i] < scores[j]:
                        concordant += 1
                elif event[i] != event[j]:
                    censored = j
                    uncensored = i
                    
                    if event[i] == 0:
                        censored = i 
                        uncensored = j
                        
                    if y_true[uncensored] <= y_true[censored]:
                        permissible += 1
                        
                        if scores[uncensored] == scores[censored]:
                            ties += 1
                        
                        if scores[uncensored] > scores[censored]:
                            concordant += 1
                
                
    result = (concordant + 0.5*ties) / permissible

    return result

In [33]:
scores = cph.predict_partial_hazard(one_hot_train)
cox_train_scores = harrell_c(one_hot_train['time'].values, scores.values, one_hot_train['status'].values)

scores = cph.predict_partial_hazard(one_hot_val)
cox_val_scores = harrell_c(one_hot_val['time'].values, scores.values, one_hot_val["status"].values)

scores = cph.predict_partial_hazard(one_hot_test)
cox_test_scores = harrell_c(one_hot_test['time'].values, scores.values, one_hot_test["status"].values)

print("Train:", cox_train_scores)
print("Val:", cox_val_scores)
print("Test:", cox_test_scores)

Train: 0.8598884066955983
Val: 0.7301587301587301
Test: 0.8231075697211155


### Random Survival Forest

In [36]:
from rpy2.robjects.packages import importr

base = importr('base')

utils = importr("utils")

import rpy2.robjects.packages as rpackages

# forest = rpackages.importr('randomForestSRC', lib_loc='R')

# from rpy2 import robjects as ro
# R = ro.r

# from rpy2.robjects import pandas2ri
# pandas2ri.activate()